In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [14]:
from google.colab import drive
drive.mount("/content/gdrive")
dataset_dir = "/content/gdrive/My Drive/Colab Notebooks/"
data = pd.read_csv(dataset_dir+'heart.csv')
data.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [15]:
catagorialList = ['sex','cp','fbs','restecg','exang','ca','thal']
for item in catagorialList:
    data[item] = data[item].astype('object')

In [16]:
data = pd.get_dummies(data, drop_first=True)

In [17]:
y = data['target'].values
y = y.reshape(y.shape[0],1)
X = data.drop(['target'],axis=1)

X.shape

(303, 21)

In [18]:
minx = np.min(X)
maxx = np.max(X)
X = (X - minx) / (maxx - minx)
X.head()

,age,trestbps,chol,thalach,oldpeak,slope,sex_1,cp_1,cp_2,cp_3,...,restecg_1,restecg_2,exang_1,ca_1,ca_2,ca_3,ca_4,thal_1,thal_2,thal_3
0,0.111702,0.257092,0.413121,0.265957,0.004078,0.000000,0.001773,0.000000,0.000000,0.001773,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.001773,0.000000,0.0
1,0.065603,0.230496,0.443262,0.331560,0.006206,0.000000,0.001773,0.000000,0.001773,0.000000,...,0.001773,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0
2,0.072695,0.230496,0.361702,0.304965,0.002482,0.003546,0.000000,0.001773,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0
3,0.099291,0.212766,0.418440,0.315603,0.001418,0.003546,0.001773,0.001773,0.000000,0.000000,...,0.001773,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0
4,0.101064,0.212766,0.627660,0.289007,0.001064,0.003546,0.000000,0.000000,0.000000,0.000000,...,0.001773,0.0,0.001773,0.0,0.0,0.0,0.0,0.000000,0.001773,0.0


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
learning_rates = [0.01, 0.1]
epochs = [50, 100]
batch_sizes = [16, 32]
neurons = [64, 128]
# Tabulate the experimental design
results = []

In [21]:
for lr in learning_rates:
    for epoch in epochs:
        for batch_size in batch_sizes:
            for neuron in neurons:
                # Define the Keras model
                model = Sequential()
                model.add(Dense(neuron, input_dim=X_train.shape[1], activation='relu'))
                model.add(Dense(1, activation='sigmoid'))

                # Compile the model
                optimizer = Adam(learning_rate=lr)
                model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

                # Train the model
                model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)

                # Evaluate the model
                _, accuracy = model.evaluate(X_test, y_test, verbose=0)

                # Record the results
                results.append({
                    'learning_rate': lr,
                    'epochs': epoch,
                    'batch_size': batch_size,
                    'neurons': neuron,
                    'accuracy': accuracy
                })

In [22]:
results_df = pd.DataFrame(results)
print(results_df)

    learning_rate  epochs  batch_size  neurons  accuracy
0            0.01      50          16       64  0.852459
1            0.01      50          16      128  0.852459
2            0.01      50          32       64  0.819672
3            0.01      50          32      128  0.819672
4            0.01     100          16       64  0.852459
5            0.01     100          16      128  0.868852
6            0.01     100          32       64  0.819672
7            0.01     100          32      128  0.836066
8            0.10      50          16       64  0.868852
9            0.10      50          16      128  0.885246
10           0.10      50          32       64  0.836066
11           0.10      50          32      128  0.836066
12           0.10     100          16       64  0.524590
13           0.10     100          16      128  0.524590
14           0.10     100          32       64  0.868852
15           0.10     100          32      128  0.901639


In [23]:
# Find the best hyperparameter combination
best_result = results_df.loc[results_df['accuracy'].idxmax()]
print("Best Hyperparameter Combination:")
print(best_result)

Best Hyperparameter Combination:
learning_rate      0.100000
epochs           100.000000
batch_size        32.000000
neurons          128.000000
accuracy           0.901639
Name: 15, dtype: float64
